# Building Effective Agents (with Pydantic AI)
## Basic Workflows

In [1]:
%pip install -r requirements.txt
from IPython.display import clear_output ; clear_output()

In [2]:
from util import initialize, show
AI_MODEL = initialize()

import asyncio
from typing import List, Dict

from pydantic import BaseModel, Field
from pydantic_ai import Agent

Available AI models:
['openai:gpt-4o',
 'openai:gpt-4o-mini',
 'gemini-1.5-pro',
 'gemini-2.0-flash-exp',
 'claude-3-5-haiku-latest',
 'claude-3-5-sonnet-latest']

Using AI model: openai:gpt-4o


### Workflow: Prompt Chaining

In [3]:
async def chain(input: str, prompts: List[str]) -> str:
    """Chain multiple LLM calls sequentially, passing results between steps."""
    agent = Agent(AI_MODEL)
    result = input
    for i, prompt in enumerate(prompts, 1):
        agent = Agent(AI_MODEL, system_prompt=prompt)
        result = (await agent.run(f"nInput:\n{result}")).data
        show(result, title=f"Step {i}")
    return result

In [4]:
data_processing_steps = [
    """Extract only the numerical values and their associated metrics from the text.
    Format each as 'value: metric' on a new line.
    Example format:
    92: customer satisfaction
    45%: revenue growth""",
    
    """Convert all numerical values to percentages where possible.
    If not a percentage or points, convert to decimal (e.g., 92 points -> 92%).
    Keep one number per line.
    Example format:
    92%: customer satisfaction
    45%: revenue growth""",
    
    """Sort all lines in descending order by numerical value.
    Keep the format 'value: metric' on each line.
    Example:
    92%: customer satisfaction
    87%: employee satisfaction""",
    
    """Format the sorted data as a markdown table with columns:
    | Metric | Value |
    |:--|--:|
    | Customer Satisfaction | 92% |"""
]

report = """
Q3 Performance Summary:
Our customer satisfaction score rose to 92 points this quarter.
Revenue grew by 45% compared to last year.
Market share is now at 23% in our primary market.
Customer churn decreased to 5% from 8%.
New user acquisition cost is $43 per user.
Product adoption rate increased to 78%.
Employee satisfaction is at 87 points.
Operating margin improved to 34%.
""".strip()

show(report, title="Input text")
formatted_result = await chain(report, data_processing_steps)
show(formatted_result, title="Result")


Input text
----------

Q3 Performance Summary:
Our customer satisfaction score rose to 92 points this quarter.
Revenue grew by 45% compared to last year.
Market share is now at 23% in our primary market.
Customer churn decreased to 5% from 8%.
New user acquisition cost is $43 per user.
Product adoption rate increased to 78%.
Employee satisfaction is at 87 points.
Operating margin improved to 34%.


Step 1
------

92: customer satisfaction  
45%: revenue growth  
23%: market share  
5%: customer churn  
$43: new user acquisition cost  
78%: product adoption rate  
87: employee satisfaction  
34%: operating margin


Step 2
------

92%: customer satisfaction  
45%: revenue growth  
23%: market share  
5%: customer churn  
43: new user acquisition cost  
78%: product adoption rate  
87%: employee satisfaction  
34%: operating margin


Step 3
------

92%: customer satisfaction  
87%: employee satisfaction  
78%: product adoption rate  
45%: revenue growth  
34%: operating margin  
23%: mar

### Workflow: Routing

In [5]:
class RouteSelection(BaseModel):
    reasoning: str = Field(..., description=(
        'Brief explanation of why this ticket should be routed '
        'to a specific team. Consider key terms, user intent, '
        'and urgency level.'
    ))
    selection: str = Field(..., description='The chosen team name')


async def route(input: str, routes: Dict[str, str]) -> str:
    """Route input to specialized prompt using content classification."""

    # First, determine appropriate route using LLM with chain-of-thought
    show(f"{list(routes.keys())}", title="Available Routes")
    
    routing_agent = Agent(
        AI_MODEL,
        system_prompt=(
            'Analyze the input and select the most appropriate support team '
            f'from these options: {list(routes.keys())}'
        ),
        result_type=RouteSelection,
    )
    route_response = await routing_agent.run(input)
    reasoning = route_response.data.reasoning
    route_key = route_response.data.selection.strip().lower()
    
    show(reasoning, title="Routing Analysis")
    show(f"{route_key}", title="Selected Route")
    
    # Process input with selected specialized prompt
    worker_agent = Agent(AI_MODEL, system_prompt=routes[route_key])
    return (await worker_agent.run(input)).data

In [6]:
support_routes = {
    "billing": """You are a billing support specialist. Follow these guidelines:
    1. Always start with "Billing Support Response:"
    2. First acknowledge the specific billing issue
    3. Explain any charges or discrepancies clearly
    4. List concrete next steps with timeline
    5. End with payment options if relevant
    
    Keep responses professional but friendly.""",
    
    "technical": """You are a technical support engineer. Follow these guidelines:
    1. Always start with "Technical Support Response:"
    2. List exact steps to resolve the issue
    3. Include system requirements if relevant
    4. Provide workarounds for common problems
    5. End with escalation path if needed
    
    Use clear, numbered steps and technical details.""",
    
    "account": """You are an account security specialist. Follow these guidelines:
    1. Always start with "Account Support Response:"
    2. Prioritize account security and verification
    3. Provide clear steps for account recovery/changes
    4. Include security tips and warnings
    5. Set clear expectations for resolution time
    
    Maintain a serious, security-focused tone.""",
    
    "product": """You are a product specialist. Follow these guidelines:
    1. Always start with "Product Support Response:"
    2. Focus on feature education and best practices
    3. Include specific examples of usage
    4. Link to relevant documentation sections
    5. Suggest related features that might help
    
    Be educational and encouraging in tone."""
}

# Test with different support tickets
tickets = [
    """Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error. 
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to 
    submit a report by end of day.
    - John""",
    
    """Subject: Unexpected charge on my card
    Message: Hello, I just noticed a charge of $49.99 on my credit card from your company, but I thought
    I was on the $29.99 plan. Can you explain this charge and adjust it if it's a mistake?
    Thanks,
    Sarah""",
    
    """Subject: How to export data?
    Message: I need to export all my project data to Excel. I've looked through the docs but can't
    figure out how to do a bulk export. Is this possible? If so, could you walk me through the steps?
    Best regards,
    Mike"""
]

print("Processing support tickets...\n")
for i, ticket in enumerate(tickets, 1):
    show(ticket, title=f"Ticket {i}")
    response = await route(ticket, support_routes)
    show(response, title=f"Response {i}")

Processing support tickets...


Ticket 1
--------

Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error. 
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to 
    submit a report by end of day.
    - John


Available Routes
----------------

['billing', 'technical', 'account', 'product']


Routing Analysis
----------------

The user is experiencing issues accessing their account and needs urgent assistance to submit a report. The problem is related to login credentials, which typically falls under account management support rather than technical issues with the website or application itself. Hence, the account support team is best equipped to handle password and access-related problems.


Selected Route
--------------

account


Response 1
----------

Account Support Response:

To address your issue with accessing your account, please follow these ste

### Workflow: Parallelization

In [7]:
async def parallel(prompt: str, inputs: List[str]) -> List[str]:
    """Process multiple inputs concurrently with the same prompt."""
    agent = Agent(AI_MODEL, system_prompt=prompt)
    results = await asyncio.gather(*[
        agent.run(input)
        for input in inputs
    ])
    return [result.data for result in results]

In [8]:
stakeholders = [
    """Customers:
    - Price sensitive
    - Want better tech
    - Environmental concerns""",
    
    """Employees:
    - Job security worries
    - Need new skills
    - Want clear direction""",
    
    """Investors:
    - Expect growth
    - Want cost control
    - Risk concerns""",
    
    """Suppliers:
    - Capacity constraints
    - Price pressures
    - Tech transitions"""
]

impact_results = await parallel(
    """Analyze how market changes will impact this stakeholder group.
    Provide specific impacts and recommended actions.
    Format with clear sections and priorities.""",
    stakeholders
)

for stakeholder, result in zip(stakeholders, impact_results):
    show(result, stakeholder.split(':')[0])


Customers
---------

### Market Changes Impact on Customers

**1. Price Sensitivity**

**Impacts:**
- **Inflation and Rising Costs:** As inflation continues to affect the economy, prices of goods, including tech products, may increase. This will directly impact price-sensitive customers who may reduce spending or switch to cheaper alternatives.
- **Shift to Affordable Brands:** High prices could push customers towards brands or retailers that offer competitive pricing, discounts, or value-added services.

**Recommended Actions:**
- **Dynamic Pricing Strategies:** Implement dynamic pricing models that adjust in real-time based on demand, ensuring competitive pricing.
- **Promotions and Discounts:** Regularly offer promotions and discounts to attract price-sensitive customers and maintain loyalty.
- **Bundle Offers:** Create product bundles that offer more value for money, encouraging upsell opportunities.

**2. Desire for Better Technology**

**Impacts:**
- **Demand for Innovation:** C